In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def pad_and_concat(sequences):  # sequences shape: [batch_size, len, dims...] -> ([batch_size, maxlen, dims...], [len])
    arrays = [np.asarray(seq) for seq in sequences]
    lengths = np.asarray([array.shape[0] for array in arrays], dtype=np.int32)
    maxlen = np.max(lengths)
    arrays = [np.pad(array, [(0, maxlen - array.shape[0]), (0, 0)], 'constant', constant_values=0) for array in arrays]
    return np.asarray(arrays), lengths
    

In [20]:
small_cnf, small_lengths = pad_and_concat(
    [[[1, -2], [2, 1]],
     [[-2, -1], [1, -2]],
     [[-1, -1], [-2, -2], [-1, -2]],
      [[1, -1]],
      [[1, -1], [1, 2]],
      [[-2, 2]]
     ])
small_sol = np.asarray([1, 1, 1, 0, 0, 0])

In [4]:
print(cnf)
print(lengths)

[[[ 1 -2]
  [ 2  1]
  [ 0  0]]

 [[-2 -1]
  [ 1 -2]
  [ 0  0]]

 [[-1 -1]
  [-2 -2]
  [-1 -2]]

 [[ 1 -1]
  [ 0  0]
  [ 0  0]]

 [[ 1 -1]
  [ 1  2]
  [ 0  0]]

 [[-2  2]
  [ 0  0]
  [ 0  0]]]
[2 2 3 1 2 1]


In [5]:
VARIABLE_NUM = 3
EMBEDDING_SIZE = 8
CLAUSE_SIZE = 2
LSTM_STATE_SIZE = 8
batch_size = 6

In [6]:
def assert_shape(matrix, shape: list):
    act_shape = matrix.get_shape().as_list()
    assert act_shape == shape, "got shape {}, expected {}".format(act_shape, shape)

In [7]:
class Graph:
    def __init__(self):
        self.inputs = tf.placeholder(tf.int32, shape=(batch_size, None, CLAUSE_SIZE), name='inputs')
        self.lengths = tf.placeholder(tf.int32, shape=(batch_size,), name='lengths')
        self.labels = tf.placeholder(tf.float32, shape=(batch_size,), name='labels')
        
        vars_ = tf.abs(self.inputs)
        signs = tf.cast(tf.sign(self.inputs), tf.float32)  # shape: [batch_size, None, CLAUSE_SIZE]

        embeddings = tf.Variable(tf.random_uniform([VARIABLE_NUM + 1, EMBEDDING_SIZE], -1., 1), name='embeddings')

        var_embeddings = tf.nn.embedding_lookup(embeddings, vars_)
        # var_embeddings shape: [None, None, CLAUSE_SIZE, EMBEDDING_SIZE]
        
        clause_preembeddings = tf.concat(
            [tf.reshape(var_embeddings, [batch_size, -1, CLAUSE_SIZE * EMBEDDING_SIZE]), 
             signs],
            axis=2)
        
        PREEMBEDDING_SIZE = EMBEDDING_SIZE * CLAUSE_SIZE + CLAUSE_SIZE
        assert_shape(clause_preembeddings, 
                     [batch_size, None, PREEMBEDDING_SIZE])
        
        clause_w = tf.Variable(tf.random_normal(
            [PREEMBEDDING_SIZE, EMBEDDING_SIZE]), name='clause_w')
        clause_b = tf.Variable(tf.random_normal([EMBEDDING_SIZE]), name='clause_b')
        clause_embeddings = tf.reshape(tf.sigmoid(
            tf.reshape(clause_preembeddings, [-1, PREEMBEDDING_SIZE]) @ clause_w + clause_b), 
                                       [batch_size, -1, EMBEDDING_SIZE])
        # shape: [None, None, EMBEDDING_SIZE]
        
        lstm = tf.contrib.rnn.BasicLSTMCell(LSTM_STATE_SIZE)
        hidden_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        current_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        state = hidden_state, current_state
        
        _, lstm_final_state = tf.nn.dynamic_rnn(lstm, clause_embeddings, dtype=tf.float32, 
                                               sequence_length=self.lengths
                                               )
        formula_embedding = lstm_final_state.h
            
        assert_shape(formula_embedding, [batch_size, LSTM_STATE_SIZE])
            
        softmax_w = tf.Variable(tf.random_normal([LSTM_STATE_SIZE, 1]), name='softmax_w')
        softmax_b = tf.Variable(tf.random_normal([1]), name='softmax_b')
        
        self.logits = tf.squeeze(formula_embedding @ softmax_w, axis=1) + softmax_b
        self.loss = tf.losses.sigmoid_cross_entropy(self.labels, self.logits) 
        self.probabilities = tf.sigmoid(self.logits)

tf.reset_default_graph()
model = Graph()

In [8]:
np.set_printoptions(precision=2, suppress=True)

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    probs = sess.run([model.probabilities], feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    
print(probs)

[array([0.44, 0.42, 0.36, 0.46, 0.35, 0.37], dtype=float32)]


In [22]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        sess.run(train_op, feed_dict={
            model.inputs: small_cnf,
            model.labels: small_sol,
            model.lengths: small_lengths
        })
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    
probs

array([1., 1., 1., 0., 0., 0.], dtype=float32)

In [11]:
import cnf

In [12]:
def sat_array():
    for k in range(1, 6):
        for var_num in range(2, 5):
            for clause_num in range(3, 10):
                sat = {True: 0, False: 0}
                for _ in range(100):
                    sat[cnf.get_random_kcnf(k, var_num, clause_num).satisfiable()] += 1
                print(k, var_num, clause_num, sat)
# sat_array()

In [41]:
cnfs = cnf.get_random_kcnfs(3000, 2, 3, 10)

In [42]:
labels = [cnf.satisfiable() for cnf in cnfs]

In [43]:
print("sat:", sum(1 for label in labels if label == True), "unsat:", sum(1 for label in labels if label == False))

sat: 2262 unsat: 738


In [44]:
def chunks(lists, chunk_size):
    return [[it[i:i + chunk_size] for it in lists] for i in range(0, len(lists[0]), chunk_size)]

In [45]:
chunks([list(range(10)), list(range(10, 20))], 2)

[[[0, 1], [10, 11]],
 [[2, 3], [12, 13]],
 [[4, 5], [14, 15]],
 [[6, 7], [16, 17]],
 [[8, 9], [18, 19]]]

In [46]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    
    for epoch_num in range(100):
        print("Epoch", epoch_num)
        losses = []
        accs = []
        for (batch_cnfs, batch_labels) in chunks((cnfs, labels), batch_size):
            if len(batch_cnfs) < batch_size:
                print("skipping incomplete batch")
                continue
            inputs, lengths = pad_and_concat([cnf.clauses for cnf in batch_cnfs])
            _, loss, probs = sess.run([train_op, model.loss, model.probabilities], feed_dict={
                model.inputs: inputs,
                model.labels: batch_labels,
                model.lengths: lengths
            })
            losses.append(loss)
            accs.append(1 - np.mean(np.abs(np.around(probs) - np.asarray(batch_labels))))
            
        print("loss:", np.mean(np.asarray(losses)), "acc:", np.mean(accs))

    print("first batch:")
    first_inputs, first_lengths = pad_and_concat([cnf.clauses for cnf in cnfs[:batch_size]])
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: first_inputs,
        model.lengths: first_lengths
    })
    print("actual:", probs)
    print("expected", labels[:batch_size])
        
    print("new data:")
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    print("actual", probs)
    print("expected", small_sol)


Epoch 0
loss: 0.45022786 acc: 0.7686666615307332
Epoch 1
loss: 0.41379803 acc: 0.7819999949336052
Epoch 2
loss: 0.40004006 acc: 0.7903333284556866
Epoch 3
loss: 0.39044702 acc: 0.7943333284556866
Epoch 4
loss: 0.3826961 acc: 0.7999999952912331
Epoch 5
loss: 0.3763461 acc: 0.806999995470047
Epoch 6
loss: 0.37156916 acc: 0.8126666623950004
Epoch 7
loss: 0.36469764 acc: 0.8159999957680703
Epoch 8
loss: 0.35149136 acc: 0.82533332914114
Epoch 9
loss: 0.3373567 acc: 0.8356666626632213
Epoch 10
loss: 0.32956094 acc: 0.837333329319954
Epoch 11
loss: 0.32416487 acc: 0.8443333294391632
Epoch 12
loss: 0.32000428 acc: 0.8396666627526284
Epoch 13
loss: 0.31467658 acc: 0.8399999959766865
Epoch 14
loss: 0.30926198 acc: 0.8453333293795585
Epoch 15
loss: 0.30446526 acc: 0.8493333294689656
Epoch 16
loss: 0.30003956 acc: 0.8506666627824306
Epoch 17
loss: 0.30086023 acc: 0.8479999961256981
Epoch 18
loss: 0.29563805 acc: 0.8563333294987678
Epoch 19
loss: 0.2944502 acc: 0.8526666627824306
Epoch 20
loss: 0.2